In [ ]:
%autosave 0

In [ ]:
#!wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5

In [ ]:
!python3 -V

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

In [ ]:
url = 'https://img.freepik.com/free-photo/winter-hiking-adventure-majestic-frozen-cliff-generative-ai_188544-12613.jpg?w=1380&t=st=1695997351~exp=1695997951~hmac=4f0ab7ac7bd59227e0f2c5ba977b35eeed35c52a8260d1efd89b4de734b8a8ac'

In [ ]:
!wget 'https://img.freepik.com/free-photo/fantastic-seascape-with-ripples_1232-424.jpg?w=1380&t=st=1695997484~exp=1695998084~hmac=8820f9661414ed450d774bb09de04c0c7d2ada0ed9dd4a546871a7c6fabdc97d' -O a.jpg

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
model = keras.models.load_model('xception2_08_0.9283.h5')

In [ ]:
img = load_img('a.jpg', target_size=(299, 299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [ ]:
img

In [ ]:
preds = model.predict(X)

In [ ]:
preds

In [ ]:
classes = [
    'buildings',
    'forest',
    'glacier',
    'mountain',
    'sea',
    'street'
]

In [ ]:
dict(zip(classes, preds[0]))

## Convert Keras to TF-Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('intel-classification-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [ ]:
!ls -lh

In [ ]:
import tensorflow.lite as tflite

In [ ]:
interpreter = tflite.Interpreter(model_path='intel-classification-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
classes = [
    'buildings',
    'forest',
    'glacier',
    'mountain',
    'sea',
    'street'   
]

dict(zip(classes, preds[0]))

## Removing TF dependency

In [ ]:
from PIL import Image

In [ ]:
with Image.open('a.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

In [ ]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [ ]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
classes = [
   'buildings',
    'forest',
    'glacier',
    'mountain',
    'sea',
    'street'
]

dict(zip(classes, preds[0]))

## Simpler way of doing it

In [7]:
!pip install keras-image-helper

You should consider upgrading via the '/home/ubuntu/.pyenv/versions/3.8.10/bin/python3.8 -m pip install --upgrade pip' command.


In [8]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
You should consider upgrading via the '/home/ubuntu/.pyenv/versions/3.8.10/bin/python3.8 -m pip install --upgrade pip' command.


In [9]:
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [10]:
interpreter = tflite.Interpreter(model_path='intel-classification-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [11]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [12]:
url = 'https://img.freepik.com/free-photo/winter-hiking-adventure-majestic-frozen-cliff-generative-ai_188544-12613.jpg?w=1380&t=st=1695997351~exp=1695997951~hmac=4f0ab7ac7bd59227e0f2c5ba977b35eeed35c52a8260d1efd89b4de734b8a8ac'
X = preprocessor.from_url(url)

In [13]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [14]:
classes = [
   'buildings',
    'forest',
    'glacier',
    'mountain',
    'sea',
    'street'
]

dict(zip(classes, preds[0]))

{'buildings': -1.6172512,
 'forest': -3.2204344,
 'glacier': 6.4395237,
 'mountain': 4.3546815,
 'sea': -6.080748,
 'street': -5.3104477}